In [1]:
import numpy as np
import numpy.linalg as alg
import sympy
import scipy.linalg as salg
import pandas as pd
import time

## Here we want to emplement a standart gradient discend method for our supply and use tables. 
(For more details please refer to a russian book "МЕТОДЫ ОПТИМИЗАЦИИ И ИССЛЕДОВАНИЕ ОПЕРАЦИЙ ДЛЯ БАКАЛАВРОВ ИНФОРМАТИКИ" by Б.А. Гладких part2). In particular we used so called proximal gradient methods.

In [2]:
def find_actives(used, X):
    '''This function is ment to find all the active restrictions'''
    activ = []
    for i in range(X.shape[0]):
        if i in used:
            continue
        if abs(-X[i] - 1) < 1e-10:
            activ.append(i)
    return set(activ)

In [3]:
def predict(previous_year, current_year, eps = 1e-8):
    D = previous_year
    B = current_year
    
    m = D.shape[1]
    n = D.shape[0]
    u = np.sum(B, axis=1)  # Summ by rows
    v = np.sum(B, axis=0)  # Summ by columns
    
    C = np.eye(D.shape[0]*D.shape[1]) * D.flatten()  # Quadratic matrix in transition from matrices to vectors
    A = np.zeros((D.shape[0]+D.shape[1], D.shape[0]*D.shape[1]))  # Restriction matrix Ax  = b
    for i in range(n):
        for j in range(i*m, (i+1)*m):
            A[i][j] = C[j][j]
    for i in range(n, n+m):
        for j in range(i-n, n*m, m):
            A[i][j] = C[j][j]
    
    M = np.eye(n*m, n*m) * -1
    b = np.array([1]*(n*m))
    used = set()  # A set of previously considred active restrictions.
    
    
    u_v = np.concatenate((u, v), axis=0)
    new_u_v = u_v - A.dot(np.ones(A.shape[1]))  # Vectors union (и and v) and modifications of our restrictions into Ax = u_v
    X = alg.pinv(A).dot(new_u_v)  # Initial approximation
    
    actives = find_actives(used, X)
    for i in actives:
        A = np.concatenate((A, [M[i]]), axis=0)
    used = used.union(actives)
    
    
    P = np.eye(A.shape[1]) - alg.pinv(A).dot(A)  # Projection matrix
    
    a_abs = np.abs(C.diagonal())  # A matrix of a square form at transition from matrices to vectors 
                                  # (to be more precise, its diagonal, we should not forget that in 
                                  #  our function these are modules |a|).
    f_val = np.sum(a_abs * X**2)  # Target function value under initial approximation.
    while 1:
        
        d = (-1)*P.dot(2*a_abs*X)  # Antigradient direction, projected to the permissible range
        if alg.norm(d) < 1e-8:
            break
        
        T = 10**20  # We want to make T as big as possible to avoid any mistakes
        for i in range(M.shape[0]):
            if i in used:
                continue
            if M[i].dot(d) != 0:
                new_T = (b[i] - M[i].dot(X)) / M[i].dot(d)
                T = min(T, new_T)
            
        T = max(0, T)
        t = - d.dot(C.dot(X)) / d.dot(C.dot(d))  # Optimal stride value
        t = min(t, T)
        
        new_X = X + t*d
        new_f = np.sum(a_abs * new_X**2)
        if abs((f_val/new_f) - 1) < eps:
            X = new_X
            break
        X = new_X
        f_val = new_f
        
        actives = find_actives(used, X)
        for i in actives:
            A = np.concatenate((A, [M[i]]), axis=0)
        used = used.union(actives)

        P = np.eye(A.shape[1]) - alg.pinv(A).dot(A)  # Projection matrix refreshment
        
        
        
    return ((X + 1) * C.diagonal()).reshape(B.shape[0], B.shape[1])

## Metrics

In [4]:
def MAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(true[i][j])
            elif result[i][j] != 0:
                answer += abs(result[i][j] - true[i][j]) / abs(result[i][j])
    return 100*answer / (true.shape[0] * true.shape[1])

In [5]:
def WAPE(result, true):
    answer = 0
    for i in range(true.shape[0]):
        for j in range(true.shape[1]):
            if true[i][j] != 0:
                answer += (abs(true[i][j]) / np.sum(true)) * abs(result[i][j] - true[i][j]) / abs(true[i][j])
            else:
                answer += abs(result[i][j] - true[i][j]) / np.sum(true)
    return answer * 100

In [6]:
def SWAD(result, true):
    return np.sum(np.abs(true*(result - true)))  / np.sum(np.square(true))

## Full circle of predictions

In [7]:
sup10 = np.load('data//sup10.npy')
sup11 = np.load('data//sup11.npy')
sup12 = np.load('data//sup12.npy')
use10 = np.load('data//use10.npy')
use11 = np.load('data//use11.npy')
use12 = np.load('data//use12.npy')

In [8]:
results = {key: {metric: 0 for metric in ['MAPE', 'WAPE', 'SWAD']} for key in ['sup11', 'sup12', 'use11', 'use12']}

In [9]:
pred_sup11 = predict(sup10, sup11)
pred_sup12 = predict(pred_sup11, sup12)

pred_use11 = predict(use10, use11)
pred_use12 = predict(pred_use11, use12)

In [10]:
for metric_name, metric in zip(['MAPE', 'WAPE', 'SWAD'], [MAPE, WAPE, SWAD]):
    results['sup11'][metric_name] = metric(pred_sup11, sup11)
    results['sup12'][metric_name] = metric(pred_sup12, sup12)
    results['use11'][metric_name] = metric(pred_use11, use11)
    results['use12'][metric_name] = metric(pred_use12, use12)

In [11]:
pd.DataFrame(results)

,sup11,sup12,use11,use12
MAPE,7.139003,9.194080,10.963089,13.160047
WAPE,1.666577,2.376965,6.067734,8.286170
SWAD,0.002962,0.004270,0.032906,0.051979
